In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.authenticators.password

In [ ]:
# export 
# hide

import abc
from time import sleep
from pymemri.plugin.states import RUN_USER_ACTION_NEEDED, RUN_USER_ACTION_COMPLETED

## Password Authenticator

Password Authenticator provides an easy interface to setup standard username-password authentication with 3rd party services.

Simply calling `authenticate()` should load the related account with required credentials.

Inheriting class should implement:
- get_token() that tests username-password combination or gets a new session token to be used for future calls


In [ ]:
# export
# hide

class PasswordAuthenticator(metaclass=abc.ABCMeta):

    SLEEP_INTERVAL = 1.0

    def __init__(self, client, pluginRun):
        self.client = client
        self.pluginRun = pluginRun
        self.isTest = False

    def authenticate(self):

        token = None
        account = self.pluginRun.account[0]

        if account.identifier and account.secret:
            token = self.get_token(username=account.identifier, password=account.secret)
            # if not token: existing credentials does not work anymore
        
        if not token:
            username, password = self.get_username_and_password()
            token = self.get_token(username=username, password=password)
            if token:
                self.store_username_and_password(username=username, password=password)
            # else raise an exception or set pluginrun.state=FAILED and pluginRun.message=ERROR_MESSAGE

        return token

        
    def get_username_and_password(self):
        if self.isTest:
            return 'username', 'password'
        # request username and password from the user client
        self.pluginRun.state = RUN_USER_ACTION_NEEDED
        self.pluginRun.update(self.client)
       # WAIT HERE = BLOCK
        while True:
            sleep(self.SLEEP_INTERVAL)
            self.pluginRun = self.client.get(self.pluginRun.id)
            if self.pluginRun.state == RUN_USER_ACTION_COMPLETED:
                account = self.pluginRun.account[0]
                return account.identifier, account.secret

    def store_username_and_password(self, username, password):
        account = self.pluginRun.account[0]
        account.identifier = username,
        account.secret = password
        account.update(self.client)

    @abc.abstractmethod
    def get_token(self, username, password):
        """ 
        A method to test username/password combination or to get a new session token for use within cookies or authentication headers 
        
        It should either return a token string or bool to indicate success
        """
        raise NotImplemented()


In [ ]:
# export 
# hide

import requests

class ExamplePasswordAuthenticator(PasswordAuthenticator):

    SERVICE_URL = "https://example.com/login"
    def get_token(self, username, password):
        result = requests.post(self.SERVICE_URL, {'username': username, 'password': password})
        if result.status_code == 200:
            try:
                res = result.json()
                return res['session_token']
            except:
                pass
